<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:100px; margin: 20px;" />
</div>

## Training and Uploading of the Model


#### We first need to import and potentially install to some libraries

Some libraries like pandas and skearn come pre installed with AI-lab. Check the error message and install the other missing libraries.
Make sure to install only the version that is stated below.

In [ ]:
%pip install xgboost==2.0.3

In [ ]:
import pandas as pd
import numpy as np
from xgboost import plot_importance, plot_tree
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

### Open Configuration


In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

### Exporting Dataset into Pandas

We have download a dataset from Kaggle called 'Store Sales Forecasting Dataset'. Uploaded it into the database and now we are importing it into this notebook.

In [ ]:
from exasol.nb_connector.connections import open_pyexasol_connection
    
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
   sql_train = f'Select * from  {ai_lab_config.db_schema}.TRAIN_TABLE'
   data = conn.export_to_pandas(sql_train)

#### Splitting the data into features (X) and target (y)


In [ ]:
X = data.drop(columns=['Sales', 'Row ID', 'Order ID', 'Customer ID', 'Customer Name', 'Product Name', 'Product ID', 'Ship Date', 'Order Date'])
y = data['Sales']

### Create train and test datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### Model Training with XGBoost
We are now executing the training of the model. This may take some time.


In [ ]:
model = XGBRegressor(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.05,
    random_state=42,
    enable_categorical=True
)
model.fit(X_train, y_train,verbose=True)

### Evaluating the model
We are now predicting with our model on the test dataset and evaluating the results with the target variable of our test dataset.
R² (R-squared) measures how well a model explains the variance in the dependent variable, with values ranging from 0 (no explanation) to 1 (perfect explanation). Values below 0 occur when the model performs worse than a simple mean model,
Our model has a R² value close to 1 which means it can be trusted with the predictions, but it doesn't guarantee a good model, especially in cases of overfitting.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Predictions
y_pred = model.predict(X_test)

# Metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.4f}")

#### Compare the results to the real sales data
With this we can see how our predicted results look like against the real world values.

In [ ]:
comparison_df = pd.DataFrame({
    'Actual_Sales': y_test,
    'Predicted_Sales': y_pred
})
print(comparison_df)

### Upload the model into BucketFS

Now that we have trained our model and tested the performance of it against our test dataset. We can now move on and upload it in BucketFS.

In [ ]:
import pickle
from exasol.nb_connector.connections import open_bucketfs_connection
from stopwatch import Stopwatch
MODEL_FILE = 'xgb_model.pkl'

# Connect to the BucketFS service
bucket = open_bucketfs_connection(ai_lab_config)

# Serialize the model into a byte-array and upload it to the BucketFS, 
# where it will be saved in the file with the specified name.
bucket.upload(MODEL_FILE, pickle.dumps(model))

print(f"Uploading the model done")

## Run Forecasting in the Exasol Database

The code connects to an Exasol database, retrieves data from a test table (TEST_TABLE), and uses a user-defined function (UDF) called XGBOOST_PRED to generate predictions. The UDF is defined within the Exasol database and performs the prediction using an XGBoost model. The model's file path (bfs_model_path) is passed as a parameter to the UDF, along with the row ID and the list of feature columns. The UDF emits the prediction results (target column Sales) for each row in the table. The predictions are then fetched and stored in a pandas DataFrame (df_pred). The process is timed using a Stopwatch to measure how long it takes to complete the predictions.
Again the `open_pyexasol_connection_with_lang_definitions()` is used so we can use our **customized SLC**.

In [ ]:
from exasol.nb_connector.slc import ScriptLanguageContainer
slc = ScriptLanguageContainer(secrets=ai_lab_config, name="sample_slc_6")
print(slc.name)

In [ ]:
from exasol.nb_connector.language_container_activation import open_pyexasol_connection_with_lang_definitions

conn = open_pyexasol_connection_with_lang_definitions(ai_lab_config, schema=ai_lab_config.db_schema, compression=True)

rs=conn.execute("""select execute_shell_command_py3('python3 -m pip list', true)""")
for r in rs: 
    print(r[0])

In [ ]:

from exasol.nb_connector.language_container_activation import open_pyexasol_connection_with_lang_definitions
from exasol.nb_connector.connections import open_pyexasol_connection, get_udf_bucket_path
from stopwatch import Stopwatch

target_column = 'Predicted Sales'
bfs_model_path = get_udf_bucket_path(ai_lab_config) + '/xgb_model.pkl'
params = {'schema': ai_lab_config.db_schema, 'test_table': 'TEST_TABLE', 'model_path': bfs_model_path}
sql = 'SELECT * FROM {schema!i}.{test_table!i} LIMIT 1'

stopwatch = Stopwatch()
conn_lang = open_pyexasol_connection_with_lang_definitions(ai_lab_config, schema=ai_lab_config.db_schema, compression=True)
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    # conn.execute(alter_session)
    # Get the list of feature columns
    df_tmp = conn.export_to_pandas(query_or_table=sql, query_params=params)
    params['column_names'] = [f'[{c}]' for c in df_tmp.columns]
    
#     # Get the predictions for all rows in the TEST table calling the prediction UDF.
#     # Provide the model path and the row ID in the first two parameters.
    sql = f'SELECT {{schema!q}}.XGBOOST({{model_path!s}},ROWID, {{column_names!r}}) ' \
        f'emits ([{target_column}] FLOAT) FROM {{schema!q}}.{{test_table!q}}'
    df_pred = conn_lang.export_to_pandas(query_or_table=sql, query_params=params)
   
    
print(f"Getting predictions took: {stopwatch}")


**Print** the predicted values and **Congratulations!** you have succesfully ran prediction on the Exasol database using Exasol's AI architecture.

In [ ]:
print(df_pred)